In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import time
from urllib.parse import urljoin
import pandas as pd

In [ ]:
response = requests.get('https://ekstrabladet.dk/nyheder/politik/')

eb_html = response.content

ebsoup = bs(eb_html, "html.parser")

In [ ]:
section_soup = ebsoup.find('div', class_ = 'flex flex-wrap--wrap flex-justify--between') # Sektion uden mest læste og top
link_soups = section_soup.find_all('a') # Links

In [ ]:
# Links og headlines - kondenseret

links = []
headlines = []

for link_soup in link_soups:
    links.append(link_soup['href'])
    headlines.append(link_soup.find('h2').get_text(strip = True))

In [ ]:
def eb_article_scrape(url):
    article_dict = {}
    
    response = requests.get(url)
    html = response.content
    artsoup = bs(html, "html.parser")
    
    artdate = artsoup.find('span', class_ = "article-timestamp--top").get_text(strip = True)
    artdate2 = artsoup.find('meta', attrs = {'property': 'og:article:published_time'})['content']
    
    try:
        ncomments = artsoup.find('span', id = 'fnTalkCommentText').get_text()
        ncomments_int = int(re.sub(r'\s.*', '', ncomments))
    except AttributeError: 
        ncomments_int = np.nan
    
    try:
        extlinks = [extlink_soup['href'] for extlink_soup in artsoup.find('div', class_ = 'article-bodytext').find_all('a')]
    except AttributeError:
        extlinks = ""
    
    try:
        author = artsoup.find('span', attrs = {'itemprop': 'author'}).get_text(strip = True)
    except AttributeError:
        author = ""
    
    article_dict['source_code'] = str(artsoup)
    article_dict['artdate_printed'] = artdate
    article_dict['artdate_meta'] = artdate2
    article_dict['ncomments'] = ncomments_int
    article_dict['extlinks'] = extlinks
    article_dict['author'] = author
    article_dict['article_url'] = url
    
    return(article_dict)

In [ ]:
new_links = [urljoin("https://ekstrabladet.dk", link) for link in links]

In [ ]:
articles = []

for c, link in enumerate(new_links, start = 1):
    articles.append(eb_article_scrape(link))
    
    print("{:.2f}%".format(100.0 * c/len(new_links)), end = '\r')
    
    time.sleep(2)

In [ ]:
data = pd.DataFrame.from_records(articles)